Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [29]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import (
    Agent,
    Runner,
    trace,
    function_tool,
    OpenAIChatCompletionsModel,
    input_guardrail,
    GuardrailFunctionOutput,
)
from typing import Dict
from sendgrid import SendGridAPIClient
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [30]:
load_dotenv(override=True)

True

In [31]:
def validate_api_key(key_name):
    key = os.getenv(key_name)
    if key:
        print(f"✓ {key_name} is exists and starts with {key[:8]}")
        return key
    else:
        print(f"⚠️ {key_name} does not exists")
        return None

In [32]:
gemini_api_key = validate_api_key("GEMINI_API_KEY") if not None else None
groq_api_key = validate_api_key("GROQ_API_KEY") if not None else None

✓ GEMINI_API_KEY is exists and starts with AIzaSyAy
✓ GROQ_API_KEY is exists and starts with gsk_9FR6


In [33]:
GEMINI_BASE_URL = validate_api_key("GEMINI_URL") if not None else None
GROQ_BASE_URL = validate_api_key("GROQ_URL") if not None else None

✓ GEMINI_URL is exists and starts with https://
✓ GROQ_URL is exists and starts with https://


In [34]:
GEMINI_MODEL = validate_api_key("GEMINI_MODEL") if not None else None
GROQ_MODEL_LLAMA = validate_api_key("GROQ_MODEL_LLAMA") if not None else None
GROQ_MODEL_DEEPSEEK = validate_api_key("GROQ_MODEL_DEEPSEEK") if not None else None

✓ GEMINI_MODEL is exists and starts with gemini-2
✓ GROQ_MODEL_LLAMA is exists and starts with meta-lla
✓ GROQ_MODEL_DEEPSEEK is exists and starts with deepseek


In [35]:
base_intro = (
    "You are an AI agent representing Anand Jain, a highly professional individual "
    "who is actively seeking new job opportunities. Your role is to write cold emails "
    "on his behalf to increase his visibility and engagement with potential employers.\n"
    "Anand's contact email: anandjain2507@gmail.com.\n"
)

instructions1 = base_intro + (
    "Your tone is formal and professional. You craft serious, respectful cold emails "
    "that convey credibility and a strong work ethic."
)

instructions2 = base_intro + (
    "Your tone is witty and engaging. You write humorous, conversational cold emails "
    "that capture attention and encourage replies."
)

instructions3 = base_intro + (
    "Your tone is efficient and direct. You write concise, no-fluff cold emails that "
    "respect the reader's time while clearly stating Anand's intent."
)

instructions4 = base_intro + (
    "Your tone is creative and story-driven. You craft imaginative cold emails using light narrative, "
    "relating Anand's experience through metaphors or anecdotes that leave a memorable impression."
)

In [36]:
groq_deepseek_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=gemini_api_key)
groq_llama_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(
    model=GROQ_MODEL_DEEPSEEK, openai_client=groq_deepseek_client
)
gemini_model = OpenAIChatCompletionsModel(
    model=GEMINI_MODEL, openai_client=gemini_client
)
llama4_model = OpenAIChatCompletionsModel(
    model=GROQ_MODEL_LLAMA, openai_client=groq_llama_client
)

In [37]:
ai_agent1 = Agent(
    name="DeepSeek AI Agent", instructions=instructions1, model=deepseek_model
)

ai_agent2 = Agent(
    name="Gemini AI Agent", instructions=instructions2, model=gemini_model
)

ai_agent3 = Agent(
    name="Llama4 AI Agent", instructions=instructions3, model=llama4_model
)

ai_agent4 = Agent(
    name="Llama4 AI Agent", instructions=instructions4, model=gemini_model
)

In [38]:
description = (
    "Compose a professional cold email to follow up on a job interview opportunity."
)

tool1 = ai_agent1.as_tool(tool_name="ai_agent1", tool_description=description)
tool2 = ai_agent2.as_tool(tool_name="ai_agent2", tool_description=description)
tool3 = ai_agent3.as_tool(tool_name="ai_agent3", tool_description=description)
tool4 = ai_agent4.as_tool(tool_name="ai_agent4", tool_description=description)

In [39]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given body to all company whose response is pending after interview completion and document submission"""
    sg = SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
    from_email = Email(
        "anandjain14314_automation.bsojl@slmail.me"
    )  # Change to your verified sender
    to_email = To("anandjain14314@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.send(mail)
    return {"status": "success"}

In [40]:
# Optimized instructions
subject_instructions = (
    "You are an expert email marketer. Given a cold sales email message, your task is to write an engaging "
    "subject line that is concise, relevant, and likely to get the recipient to open the email."
)

html_instructions = (
    "You are an email design expert. Convert the provided plain-text or markdown email body into a clean, "
    "visually appealing HTML layout. Focus on readability, clarity, and responsiveness across devices."
)

cta_instructions = (
    "You are a conversion specialist. Given the body of a cold email, craft a clear and persuasive call-to-action (CTA) "
    "that motivates the recipient to reply, book a meeting, or take the desired next step."
)

# Tool declarations
subject_writer = Agent(
    name="Email Subject Writer", instructions=subject_instructions, model=deepseek_model
)
subject_tool = subject_writer.as_tool(
    tool_name="subject_writer", tool_description="Write a subject line for a cold email"
)

html_converter = Agent(
    name="HTML Email Body Converter", instructions=html_instructions, model=gemini_model
)
html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert a plain text email body to HTML",
)

cta_writer = Agent(
    name="Call-to-Action Writer", instructions=cta_instructions, model=llama4_model
)
cta_tool = cta_writer.as_tool(
    tool_name="cta_writer",
    tool_description="Write a strong call-to-action for a cold email",
)

In [41]:
email_tools = [subject_tool, html_tool, cta_tool, send_html_email]

In [42]:
instructions = (
    "You are an intelligent email automation agent. Given the plain text body of an email:\n"
    "1. Use the `subject_writer` tool to generate a compelling subject line.\n"
    "2. Use the `html_converter` tool to convert the email body into a clean, responsive HTML layout.\n"
    "3. Use the `cta_writer` tool to generate a persuasive call-to-action to include at the end of the email.\n"
    "4. Use the `send_html_email` tool to send the final email with the generated subject, HTML body, and CTA.\n"
    "Ensure the final output is polished, professional, and ready for delivery."
)

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,  # Make sure `tools` includes all four: subject_writer, html_converter, cta_writer, send_html_email
    model=gemini_model,
    handoff_description="Generate an HTML email with a subject and CTA, then send it",
)

In [43]:
tools = [tool1, tool2, tool3, tool4]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='ai_agent1', description='Compose a professional cold email to follow up on a job interview opportunity.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001DC54648720>, strict_json_schema=True), FunctionTool(name='ai_agent2', description='Compose a professional cold email to follow up on a job interview opportunity.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001DC5671A7A0>, strict_json_schema=True), FunctionTool(name='ai_agent3', description='Compose a professional cold email to follow up on

In [44]:
followup_manager_instructions = (
    "You are the Follow-up Manager agent representing Anand Jain, a professional job applicant. "
    "Your task is to generate effective, polite, and professional follow-up emails using the provided follow-up email generation tools. "
    "You never write the follow-up email directly; instead, you must invoke the tools. "
    "You will try all three follow-up email generator tools at least once, and can call them multiple times if needed to improve the output. "
    "After generating multiple versions, you select the single best email based on which is most likely to elicit a positive response. "
    "Once selected, you hand off the chosen email to the Email Manager agent to format and send."
)

followup_manager = Agent(
    name="Follow-up Manager",
    instructions=followup_manager_instructions,
    tools=tools,  # Ensure this includes the follow-up email generators and Email Manager tools
    handoffs=handoffs,  # Email Manager agent should be part of handoffs for smooth transfer
    model=llama4_model,
)

message = (
    "Generate a professional follow-up email to the HR or recruitment team. "
    "The candidate, Anand Jain, has completed 5 interview rounds and submitted all required documents over a month ago, "
    "but has yet to receive any update. The email should be polite, concise, and express continued interest in the position."
)

with trace("Automated Candidate Follow-up"):
    result = await Runner.run(followup_manager, message)

## Check out the trace:

https://platform.openai.com/traces

GuardRails
can only be applied to either input of first agent or output of the last agent.
can not apply in between.

In [45]:
llama4_scout_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

llama4_scout_model = OpenAIChatCompletionsModel(
    model=os.getenv("GROQ_MODEL_LLAMA_SCOUT"), openai_client=llama4_scout_client
)

In [46]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str


guardrail_agent = Agent(
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model=llama4_scout_model,
)

In [47]:
@input_guardrail
async def guardrail_against_name(ctx, aganet, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(
        output_info={"found_name": result.final_output},
        tripwire_triggered=is_name_in_message,
    )

In [49]:
job_followup_manager = Agent(
    name="Job Follow-up Manager",
    instructions=followup_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],  # Make sure this includes all formatted tools
    model=llama4_scout_model,  # Use a valid, accessible model
    input_guardrails=[
        guardrail_against_name
    ],  # Reuse if applicable to job-seeking messages
)

message = (
    "Send a polite and professional follow-up email to the recruitment team. "
    "The random candidate has completed 5 interview rounds and submitted all required documents "
    "over a month ago but has not received any update. The email should express continued interest in the role."
)

with trace("Automated Candidate Follow-up"):
    result = await Runner.run(job_followup_manager, message)